# GRASS Tool with Parameters

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# % keyword: statistics
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# % guisection: Input
# %end
# %option G_OPT_V_INPUT
# % key: points
# % guisection: Input
# %end
# %option G_OPT_R_INPUTS
# % guisection: Input
# %end
# %option G_OPT_V_OUTPUT
# % guisection: Output
# %end
# %option G_OPT_M_NPROCS
# %end


import atexit
import multiprocessing
import csv
import io
import tempfile

from pathlib import Path

import grass.script as gs


def data_to_csv(results, file_name, csv_header):
    with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=csv_header, extrasaction="ignore")
        writer.writeheader()
        for point, result in results:
            row = {}
            for raster_name, statistics in result.items():
                for key, value in statistics.items():
                    new_key = f"{raster_name}_{key}"
                    point[new_key] = value
            row.update(point)
            writer.writerow(row)


def output_results(results, sample_rasters, output):
    with tempfile.TemporaryDirectory() as tmp_dir:
        file_name = Path(tmp_dir) / "data.txt"
        csv_header = ["cat", "east", "north"]
        database_columns = ["cat INTEGER", "east REAL", "north REAL"]
        for raster in sample_rasters:
            for value in ["mean", "stddev", "min", "max"]:
                name = f"{raster}_{value}"
                csv_header.append(name)
                database_columns.append(f"{name} REAL")
        data_to_csv(results, file_name, csv_header)
        gs.run_command(
            "v.in.ascii",
            input=file_name,
            output=output,
            format="point",
            separator="comma",
            cat=1,
            x=2,
            y=3,
            skip=1,
            columns=database_columns,
        )


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def multiple_viewsheds(
    elevation,
    points,
    sample_rasters,
    output,
    nprocs,
):
    point_data = gs.read_command(
        "v.out.ascii",
        input=points,
        type="point",
        format="point",
        separator="comma",
        flags="cr",
    )
    reader = csv.DictReader(io.StringIO(point_data))
    data = []
    for point in reader:
        data.append((elevation, point, sample_rasters))

    with multiprocessing.Pool(processes=nprocs) as pool:
        results = pool.starmap(parametrized_viewshed, data)
    output_results(results, sample_rasters, output)


def parametrized_viewshed(elevation, point, sample_rasters):
    coordinates = (point["east"], point["north"])
    result = one_viewshed(
        elevation=elevation,
        coordinates=coordinates,
        sample_rasters=sample_rasters,
    )
    return point, result


def one_viewshed(
    elevation,
    coordinates,
    sample_rasters,
):
    viewshed = gs.append_node_pid(
        gs.legalize_vector_name(f"tmp_viewshed_{coordinates}")
    )
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
        quiet=True,
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    return results


def main():
    options, unused_flags = gs.parser()
    sample_rasters = options["input"].split(",")
    multiple_viewsheds(
        elevation=options["elevation"],
        points=options["points"],
        sample_rasters=sample_rasters,
        output=options["output"],
        nprocs=int(options["nprocs"]),
    )


if __name__ == "__main__":
    main()

As before, we will make the script executable:

In [ ]:
!chmod u+x viewscape.py

Running the script with `--help` gives its interface described for command line use:

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation points=firestations input=elevation,ndvi output=points nprocs=4 --o 

## Check Code Quality

In [ ]:
!black --diff viewscape.py
!black viewscape.py
!flake8 --max-line-length=88 --ignore=E501 viewscape.py
!grass --tmp-location XY --exec pylint --disable=C0114,C0116 viewscape.py

## Self-study Material

* [FOSS4G 2022 workshop: Developing custom GRASS tools](https://github.com/wenzeslaus/foss4g-2022-developing-custom-grass-tools)